In [3]:
# Cell 1: Initial setup and data loading
from itertools import compress
import matplotlib.pyplot as plt
import numpy as np
import os
import mne
import mne_nirs
from mne_nirs.statistics import run_glm
import statsmodels.formula.api as smf
import pandas as pd

# Set up FreeSurfer paths
# wsl_freesurfer_home = r"\\wsl$\\Ubuntu-22.04\\usr\\local\\freesurfer\\7.4.1"
# wsl_subjects_dir = wsl_freesurfer_home + r"\\subjects"
# subjects_dir = wsl_subjects_dir
# os.environ['FREESURFER_HOME'] = wsl_freesurfer_home
# os.environ['SUBJECTS_DIR'] = wsl_subjects_dir

print("FreeSurfer environment configured for WSL")

# Function to extract trial name from filename
def extract_trial_name(snirf_file):
    filename = os.path.basename(snirf_file)
    filename_parts = filename.replace('.snirf', '').split('_')
    
    if 'Keehn' in filename_parts and 'Logan' in filename_parts:
        trial = "MRSA"
    elif 'Will' in filename_parts and 'Rudd' in filename_parts:
        trial = 'none'
    elif 'Timothy' in filename_parts and 'Yang' in filename_parts:
        trial = 'R'
    else:
        trial = 'unknown'
    
    return trial

def process_single_file(snirf_file, participant_id):
    """Process a single SNIRF file through the complete pipeline"""
    
    trial_name = extract_trial_name(snirf_file)
    print(f"Loading data from: {snirf_file} (Trial: {trial_name})")
    
    # Load and process data (following your original cells 1-12)
    raw_intensity = mne.io.read_raw_snirf(snirf_file, optode_frame='unknown', verbose="info")
    raw_intensity.load_data()
    
    # Remove thinkaloud annotations
    raw_intensity.annotations.delete([index for index,value in enumerate(raw_intensity.annotations.description) if 'thinkaloud' in value])
    
    # Convert to hemoglobin
    raw_od = mne.preprocessing.nirs.optical_density(raw_intensity, verbose=False)
    raw_haemo = mne.preprocessing.nirs.beer_lambert_law(raw_od, ppf=0.1)
    
    # Resample
    print(f"Data shape: {raw_haemo.get_data().shape}")
    print(f"Sampling frequency: {raw_haemo.info['sfreq']} Hz")
    raw_haemo.resample(4)
    
    # Create design matrix
    design_matrix = mne_nirs.experimental_design.make_first_level_design_matrix(
        raw_haemo,
        drift_model="cosine",
        high_pass=0.025,
        hrf_model="spm",
    )
    
    print(f"Design matrix shape: {design_matrix.shape}")
    print(f"Design matrix columns: {design_matrix.columns.tolist()}")
    
    # Run GLM
    glm_est = run_glm(
        raw_haemo, 
        design_matrix,
        noise_model='ar1',
        bins=0
    )
    
    print("GLM estimation completed using AR-IRLS algorithm")
    
    return raw_haemo, glm_est, design_matrix, trial_name

# Define your two files
snirf_file1 = "..\\data\\raw\\250624_1014_20250624_Keehn_Logan_NS_Keehn_Logan_concat.snirf"
snirf_file2 = "..\\data\\raw\\250625_1408_20250625_Rudd_Will_NS_rudd_will.snirf"  # Replace with your second file path

# Process both files
print("=== Processing File 1 ===")
raw_haemo1, glm_est1, design_matrix1, trial1 = process_single_file(snirf_file1, 1)

print("\n=== Processing File 2 ===")
raw_haemo2, glm_est2, design_matrix2, trial2 = process_single_file(snirf_file2, 2)

FreeSurfer environment configured for WSL
=== Processing File 1 ===
Loading data from: ..\data\raw\250624_1014_20250624_Keehn_Logan_NS_Keehn_Logan_concat.snirf (Trial: MRSA)
Loading c:\Users\jdtull\Documents\CRED\Projects\Design Cognition\Design-Cognition\cred_fnirs_toolkit\notebooks\..\data\raw\250624_1014_20250624_Keehn_Logan_NS_Keehn_Logan_concat.snirf
Found jitter of 0.000000% in sample times.
Reading 0 ... 19631  =      0.000 ...  2453.875 secs...


C:\Users\jdtull\AppData\Local\Temp\ipykernel_35064\377737358.py:51: RuntimeWarning: Negative intensities encountered. Setting to abs(x)
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity, verbose=False)


Data shape: (408, 19632)
Sampling frequency: 8.0 Hz
Design matrix shape: (9816, 128)
Design matrix columns: ['Design problem context', 'Generate stakeholder needs', 'Prioritize stakeholder needs', 'Stakeholder identification', 'Stakeholder personas', 'drift_1', 'drift_2', 'drift_3', 'drift_4', 'drift_5', 'drift_6', 'drift_7', 'drift_8', 'drift_9', 'drift_10', 'drift_11', 'drift_12', 'drift_13', 'drift_14', 'drift_15', 'drift_16', 'drift_17', 'drift_18', 'drift_19', 'drift_20', 'drift_21', 'drift_22', 'drift_23', 'drift_24', 'drift_25', 'drift_26', 'drift_27', 'drift_28', 'drift_29', 'drift_30', 'drift_31', 'drift_32', 'drift_33', 'drift_34', 'drift_35', 'drift_36', 'drift_37', 'drift_38', 'drift_39', 'drift_40', 'drift_41', 'drift_42', 'drift_43', 'drift_44', 'drift_45', 'drift_46', 'drift_47', 'drift_48', 'drift_49', 'drift_50', 'drift_51', 'drift_52', 'drift_53', 'drift_54', 'drift_55', 'drift_56', 'drift_57', 'drift_58', 'drift_59', 'drift_60', 'drift_61', 'drift_62', 'drift_63', 'd

C:\Users\jdtull\AppData\Local\Temp\ipykernel_35064\377737358.py:44: RuntimeWarning: Extraction of measurement date from SNIRF file failed. The date is being set to January 1st, 2000, instead of 2025-06-25
  raw_intensity = mne.io.read_raw_snirf(snirf_file, optode_frame='unknown', verbose="info")
C:\Users\jdtull\AppData\Local\Temp\ipykernel_35064\377737358.py:51: RuntimeWarning: Negative intensities encountered. Setting to abs(x)
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity, verbose=False)


Data shape: (408, 17541)
Sampling frequency: 8.138020833333334 Hz
Design matrix shape: (8622, 115)
Design matrix columns: ['Design problem context', 'Generate stakeholder needs', 'Prioritize stakeholder needs', 'Stakeholder identification', 'Thinkaloud practice AUT', 'Thinkaloud practice basic math', 'Thinkalound practice anagram', 'drift_1', 'drift_2', 'drift_3', 'drift_4', 'drift_5', 'drift_6', 'drift_7', 'drift_8', 'drift_9', 'drift_10', 'drift_11', 'drift_12', 'drift_13', 'drift_14', 'drift_15', 'drift_16', 'drift_17', 'drift_18', 'drift_19', 'drift_20', 'drift_21', 'drift_22', 'drift_23', 'drift_24', 'drift_25', 'drift_26', 'drift_27', 'drift_28', 'drift_29', 'drift_30', 'drift_31', 'drift_32', 'drift_33', 'drift_34', 'drift_35', 'drift_36', 'drift_37', 'drift_38', 'drift_39', 'drift_40', 'drift_41', 'drift_42', 'drift_43', 'drift_44', 'drift_45', 'drift_46', 'drift_47', 'drift_48', 'drift_49', 'drift_50', 'drift_51', 'drift_52', 'drift_53', 'drift_54', 'drift_55', 'drift_56', 'dr

In [8]:


def create_side_by_side(raw_haemo1, raw_haemo2, glm_est1, glm_est2, trial1, trial2):
    """Create side-by-side brain visualizations using your original approach"""
    
    # Create transformation matrix
    trans_matrix = np.array([
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.01],
        [0., 0., 0., 1.]
    ])
    adjusted_trans = mne.transforms.Transform(fro='head', to='mri', trans=trans_matrix)
    
    # Get conditions from both files
    conditions1 = [item for item in design_matrix1.columns.tolist() if ('drift' not in item) and ('constant' not in item)]
    conditions2 = [item for item in design_matrix2.columns.tolist() if ('drift' not in item) and ('constant' not in item)]
    
    # Find common conditions
    common_conditions = list(set(conditions1) & set(conditions2))
    print(f"Common conditions: {common_conditions}")
    print(f"Trial 1: {trial1}, Trial 2: {trial2}")
    
    # Create results directory
    os.makedirs('results/comparison', exist_ok=True)
    
    # Create visualizations for each common condition
    for condition in common_conditions:
        print(f"\nProcessing condition: {condition}")
        
        try:
            # Create brain visualizations using your original surface_projection method
            brain1 = glm_est1.copy().surface_projection(
                condition=condition,
                view="dorsal",
                chroma="hbo",
            )
            brain1.add_text(text=f"{condition} - {trial1}", x=50, y=0)
            
            brain2 = glm_est2.copy().surface_projection(
                condition=condition,
                view="dorsal",
                chroma="hbo",
            )
            brain2.add_text(text=f"{condition} - {trial2}", x=50, y=0)
            
            # Set consistent views
            brain1.show_view(view="coronal")
            brain2.show_view(view="coronal")
            
            # Save individual images
            brain1.save_image(f'results/comparison/{condition}_{trial1}.png')
            brain2.save_image(f'results/comparison/{condition}_{trial2}.png')
            
            print(f"Saved brain visualizations for {condition}")
            
        except Exception as e:
            print(f"Error processing condition {condition}: {e}")
    
    return common_conditions

# Create side-by-side visualizations
common_conditions = create_side_by_side(
    raw_haemo1, raw_haemo2, glm_est1, glm_est2, trial1, trial2
)

Common conditions: ['Generate stakeholder needs', 'Prioritize stakeholder needs', 'Stakeholder identification', 'Design problem context']
Trial 1: MRSA, Trial 2: none

Processing condition: Generate stakeholder needs
Error processing condition Generate stakeholder needs: 'Key "SUBJECTS_DIR" not found in the environment or in the the mne-python config file (C:\\Users\\jdtull\\.mne\\mne-python.json). Try either os.environ["SUBJECTS_DIR"] = VALUE for a temporary solution, or mne.utils.set_config("SUBJECTS_DIR", VALUE, set_env=True) for a permanent one. You can also set the environment variable before running python.'

Processing condition: Prioritize stakeholder needs
Error processing condition Prioritize stakeholder needs: 'Key "SUBJECTS_DIR" not found in the environment or in the the mne-python config file (C:\\Users\\jdtull\\.mne\\mne-python.json). Try either os.environ["SUBJECTS_DIR"] = VALUE for a temporary solution, or mne.utils.set_config("SUBJECTS_DIR", VALUE, set_env=True) for a 

In [6]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.gridspec import GridSpec

def create_combined_comparison_plots(common_conditions, trial1, trial2):
    """Create combined side-by-side comparison plots"""
    
    for condition in common_conditions:
        try:
            # Load the individual brain images
            img1_path = f'results/comparison/{condition}_{trial1}.png'
            img2_path = f'results/comparison/{condition}_{trial2}.png'
            
            if os.path.exists(img1_path) and os.path.exists(img2_path):
                img1 = mpimg.imread(img1_path)
                img2 = mpimg.imread(img2_path)
                
                # Create side-by-side plot
                fig = plt.figure(figsize=(16, 8))
                gs = GridSpec(1, 2, figure=fig, wspace=0.05)
                
                # Plot first image
                ax1 = fig.add_subplot(gs[0])
                ax1.imshow(img1)
                ax1.set_title(f"{condition} - {trial1}", fontsize=14, fontweight='bold')
                ax1.axis('off')
                
                # Plot second image
                ax2 = fig.add_subplot(gs[1])
                ax2.imshow(img2)
                ax2.set_title(f"{condition} - {trial2}", fontsize=14, fontweight='bold')
                ax2.axis('off')
                
                # Add main title
                fig.suptitle(f'Comparison: {condition} ({trial1} vs {trial2})', 
                           fontsize=16, fontweight='bold', y=0.95)
                
                # Save combined image
                combined_path = f'results/comparison/{condition}_comparison_{trial1}_vs_{trial2}.png'
                fig.savefig(combined_path, dpi=300, bbox_inches='tight', 
                           facecolor='white', edgecolor='none')
                plt.show()
                
                print(f"Created combined comparison for {condition}")
                
            else:
                print(f"Missing image files for condition {condition}")
                
        except Exception as e:
            print(f"Error creating combined plot for {condition}: {e}")

# Create combined comparison plots
create_combined_comparison_plots(common_conditions, trial1, trial2)

NameError: name 'common_conditions' is not defined

In [7]:

def create_summary_plot(common_conditions, trial1, trial2):
    """Create a summary overview of all conditions"""
    
    n_conditions = len(common_conditions)
    if n_conditions == 0:
        print("No common conditions found for comparison")
        return
    
    # Calculate grid dimensions
    cols = min(3, n_conditions)  # Max 3 columns
    rows = (n_conditions + cols - 1) // cols  # Ceiling division
    
    fig = plt.figure(figsize=(5*cols, 4*rows))
    
    for i, condition in enumerate(common_conditions):
        try:
            # Load combined comparison image
            img_path = f'results/comparison/{condition}_comparison_{trial1}_vs_{trial2}.png'
            
            if os.path.exists(img_path):
                img = mpimg.imread(img_path)
                
                ax = fig.add_subplot(rows, cols, i+1)
                ax.imshow(img)
                ax.set_title(condition, fontsize=12, fontweight='bold')
                ax.axis('off')
            else:
                print(f"Missing comparison image for {condition}")
                
        except Exception as e:
            print(f"Error loading image for {condition}: {e}")
    
    plt.suptitle(f'fNIRS Activation Comparison: {trial1} vs {trial2}', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save summary plot
    summary_path = f'results/comparison/summary_{trial1}_vs_{trial2}.png'
    fig.savefig(summary_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"Created summary comparison plot: {summary_path}")

# Create summary plot
create_summary_plot(common_conditions, trial1, trial2)

NameError: name 'common_conditions' is not defined